## **Deprecated** SBERT endpoint (This notebook is for a reference. We don't need to create it again.)

In [2]:
!pip install transformers

In [6]:
# Choose transformer model for embeddings
from transformers import AutoTokenizer, AutoModel
import os
import sagemaker
import time
# saved_model_dir = 'transformer'
# os.makedirs(saved_model_dir, exist_ok=True)

# tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/multi-qa-MiniLM-L6-cos-v1")
# tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
# model = AutoModel.from_pretrained("sentence-transformers/multi-qa-MiniLM-L6-cos-v1") 

# tokenizer.save_pretrained(saved_model_dir)
# model.save_pretrained(saved_model_dir)

#Defining default bucket for SageMaker pretrained model hosting
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# !cd transformer && tar czvf ../model.tar.gz *

# model_data = sagemaker_session.upload_data(path='model.tar.gz', key_prefix='autofaiss-demo/huggingface-models')


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [7]:
from sagemaker.huggingface.model import HuggingFaceModel

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
   #model_data=model_data,       # path to your model and script
   #entry_point = 'predict.py',
   #source_dir = 'source_dir',
   role=role,                    # iam role with permissions to create an Endpoint
   transformers_version="4.12",  # transformers version used
   pytorch_version="1.9",        # pytorch version used
   py_version='py38',            # python version used
   env={ 
       'HF_TASK':'feature-extraction',
       'HF_MODEL_ID': 'sentence-transformers/multi-qa-MiniLM-L6-cos-v1'
   }
)

# deploy the endpoint endpoint
predictor = huggingface_model.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.large",
    endpoint_name="sm-cc-aws"
    )

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
----!

In [1]:
## Trying out chat template but the kernel keeps crashing
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/multi-qa-MiniLM-L6-cos-v1")

chat = [
    {"role": "system", "content": "You are a Spanish language teacher, and the user made mistakes. You respond with 'ahh, you mean,...' and repeat what the user said in the correct format. Don't further explain, and keep your response in one short sentence."},
    {"role": "user", "content": "Sí, tengo algo tiempo hoy."},
]

tokenizer.use_default_system_prompt = False
tokenizer.apply_chat_template(chat, tokenize=False)


No chat template is defined for this tokenizer - using a default chat template that implements the ChatML format. If the default is not appropriate for your model, please set `tokenizer.chat_template` to an appropriate template. See https://huggingface.co/docs/transformers/main/chat_templating for more information.



"<|im_start|>system\nYou are a Spanish language teacher, and the user made mistakes. You respond with 'ahh, you mean,...' and repeat what the user said in the correct format. Don't further explain, and keep your response in one short sentence.<|im_end|>\n<|im_start|>user\nSí, tengo algo tiempo hoy.<|im_end|>\n"

In [ ]:
messages2 = [
    {"role": "system", "content": "You are a Spanish language teacher, and the user made mistakes. You respond with 'ahh, you mean,...' and repeat what the user said in the correct format. Don't further explain, and keep your response in one short sentence."},
    {"role": "user", "content": "Sí, tengo algo tiempo hoy."},
]
prompt2 = pipe.tokenizer.apply_chat_template(messages2, tokenize=False, add_generation_prompt=True)
outputs2 = pipe(prompt2, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
print(outputs2[0]["generated_text"])

In [11]:
!pip install accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.2 MB/s eta 0:00:0000:01


In [8]:
# delete endpoint
predictor.delete_endpoint()